## Feature Engineering

In [1]:
!pip install fuzzywuzzy
!pip install python-Levenshtein

In [2]:
import pandas as pd
import numpy as np
import time
import re
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [3]:
data = pd.read_csv('../../Data/Amazon/meta_Books.csv')

/var/folders/kn/5p9n18rd2r58j5lzlvq6zh040000gn/T/ipykernel_1119/2215363889.py:1: DtypeWarning: Columns (2,4,13,14,19) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../../Data/Amazon/meta_Books.csv')


In [4]:
data.head()

,Unnamed: 0,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details
0,0,[],NaN,['It is a biology book with God&apos;s perspec...,NaN,Biology Gods Living Creation Third Edition 10 ...,"['0669009075', 'B000K2P5SA', 'B00MD4G2N0', 'B0...",NaN,Keith Graham,[],"1,349,781 in Books (","['0019777701', 'B000AUCX7I', 'B000K2P5SA', 'B0...",Books,NaN,NaN,$39.94,0000092878,[],[],NaN
1,1,"['Books', 'New, Used & Rental Textbooks', 'Med...",NaN,[],NaN,Mksap 16 Audio Companion: Medical Knowledge Se...,[],NaN,Acp,[],"1,702,625 in Books (","['B01MUCYEV7', 'B01KUGTY6O']",Books,NaN,NaN,NaN,000047715X,[],[],NaN
2,2,"['Books', 'Arts & Photography', 'Music']",NaN,"['Discography of American Punk, Hardcore, and ...",NaN,"Flex! Discography of North American Punk, Hard...",[],NaN,Burkhard Jarisch,[],"6,291,012 in Books (",[],Books,NaN,NaN,$199.99,0000004545,[],[],NaN
3,3,"['Books', 'Arts & Photography', 'Music']",NaN,['This is a collection of classic gospel hymns...,NaN,Heavenly Highway Hymns: Shaped-Note Hymnal,[],NaN,Stamps/Baxter,[],"2,384,057 in Books (","['0006180116', '0996092730', 'B000QFOGY0', 'B0...",Books,NaN,NaN,NaN,0000013765,[],[],NaN
4,4,[],NaN,[],NaN,Georgina Goodman Nelson Womens Size 8.5 Purple...,[],NaN,NaN,[],"11,735,726 in Books (",[],Books,NaN,NaN,$164.10,0000000116,[],[],NaN


In [5]:
data_kaggle = pd.read_csv('../../Data/Kaggle/Books.csv')

/var/folders/kn/5p9n18rd2r58j5lzlvq6zh040000gn/T/ipykernel_1119/2966050337.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  data_kaggle = pd.read_csv('../../Data/Kaggle/Books.csv')


In [6]:
data_kaggle.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [7]:
# data_kaggle = data_kaggle.iloc[:20,:]

In [8]:
# final_ratio = {}
final_values = []
final_indices = []

start = time.time()
for book_kaggle in data_kaggle['Book-Title']:
    ratio = []
    search_space_indices = []
    i = 0
    while len(search_space_indices) == 0:
        try:
            first_word = str.split(book_kaggle)[i]
            first_word = re.sub('[^A-Za-z0-9]+', '', first_word)
#             print(first_word)
            if first_word in ['The', 'A']:
                first_word = str.split(book_kaggle)[i+1] # maybe not the best strategy
            search_space = data["title"].str.find(first_word) == 0
            search_space_indices = data['title'][search_space].reset_index().iloc[:,0]
#             if i > 0:
#                 print('Went to the next word............')
        except:
#             print(book_kaggle, '...............................................')
            search_space_indices = [1]
            break
        i = i+1
        
#     if len(search_space_indices) == 0 && search_space_indices[0] == 'empty':
        
#     print(len(search_space_indices))
    for book_amazon in data['title'][search_space_indices]:
        value = fuzz.token_sort_ratio(book_kaggle, book_amazon)
        ratio.append(value)
#     print(ratio)
    final_indices.append(search_space_indices[np.argmax(ratio)])
    final_values.append(ratio[np.argmax(ratio)])
    
end = time.time()
print("The time of execution of above program is :",
      (end-start), "s")

The time of execution of above program is : 18.16628909111023 s


In [9]:
data_amazon_sel = data.iloc[final_indices, [1,3,5,6,10,11,15,16]]
data_amazon_sel['match_value'] = final_values
data_amazon_sel.head()

,category,description,title,also_buy,rank,also_view,price,asin,match_value
120125,"['Books', 'New, Used &amp; Rental Textbooks', ...","['""A fine introduction to the subject, clear, ...",Classical Mythology,"['019953831X', '0199536325', '0199537968', '05...","5,219,563 in Books (","['019999739X', '0195308050', '0195397703', '00...",NaN,0195143388,100
20506,"['Books', ""Children's Books"", 'Animals']","['Pamela Duncan Edwards and Henry Cole, the cr...",Clara Caterpillar,[],"4,370,670 in Books (","['1782358854', '0064435024', '0064435164', '00...",$14.89,0060289953,76
1523043,"['Books', 'Literature &amp; Fiction', 'Literary']",['Darlene Biese Schultz teaches fiction writin...,Decision or Destiny,[],"10,562,636 in Books (",[],$12.99,1463527101,72
234511,"['Books', 'Health, Fitness &amp; Dieting']",[],Flu: The Story of the Great Influenza Pandmic ...,"['0143036491', 'B005APQFD6', '1594489254', '15...","10,931,958 in Books (","['0143036491', '1541736125', '0195307143', '05...",$3.80,0333751051,99
2509648,"['Books', 'Arts & Photography', 'Collections, ...","['This compelling book provides a historical, ...",Mummies of the World,[],"3,982,059 in Books (",[],$29.99,3791350307,76


In [27]:
data_amazon_sel

,category,description,title,also_buy,rank,also_view,price,asin,match_value
120125,"['Books', 'New, Used &amp; Rental Textbooks', ...","['""A fine introduction to the subject, clear, ...",Classical Mythology,"['019953831X', '0199536325', '0199537968', '05...","5,219,563 in Books (","['019999739X', '0195308050', '0195397703', '00...",NaN,0195143388,100
20506,"['Books', ""Children's Books"", 'Animals']","['Pamela Duncan Edwards and Henry Cole, the cr...",Clara Caterpillar,[],"4,370,670 in Books (","['1782358854', '0064435024', '0064435164', '00...",$14.89,0060289953,76
1523043,"['Books', 'Literature &amp; Fiction', 'Literary']",['Darlene Biese Schultz teaches fiction writin...,Decision or Destiny,[],"10,562,636 in Books (",[],$12.99,1463527101,72
234511,"['Books', 'Health, Fitness &amp; Dieting']",[],Flu: The Story of the Great Influenza Pandmic ...,"['0143036491', 'B005APQFD6', '1594489254', '15...","10,931,958 in Books (","['0143036491', '1541736125', '0195307143', '05...",$3.80,0333751051,99
2509648,"['Books', 'Arts & Photography', 'Collections, ...","['This compelling book provides a historical, ...",Mummies of the World,[],"3,982,059 in Books (",[],$29.99,3791350307,76
948392,"['Books', 'Crafts, Hobbies & Home', 'Antiques ...",[],Kitchens and Kitchenware,[],"7,078,424 in Books (",[],$6.59,0870694251,70
233538,"['Books', 'History', 'Historical Study &amp; E...",['Slight shelf wear. Pages are clean. Binding ...,What If? : Military Historians Imagine What Mi...,"['042518613X', '0425198189', '1616085460', '03...","5,592,796 in Books (","['042518613X', '0399152385', '0425198189', '16...",$2.40,0330487248,84
69694,"['Books', 'Mystery, Thriller &amp; Suspense', ...","['RUTH RENDELL has won numerous awards, includ...","GUILTY THING SURPRISED, A","['0345513584', '0345498453', '0375704892', '00...","2,025,533 in Books (","['0345342534', '0345345304', '0375704892', '03...",$17.78,0099534843,67
111790,"['Books', 'Engineering &amp; Transportation', ...","[""David Cordingly is the world's foremost expe...",Under the Black Flag: The Romance and the Real...,"['015603462X', '1479322571', '0312335806', '07...","2,552,673 in Books (","['015603462X', '1631492101', '1479322571', '19...",$10.55,0156005492,92
550179,"['Books', 'Literature &amp; Fiction', 'Short S...",['Ann Beattie is a writer of enormous compassi...,Where You'Ll Find Me: And Other Stories,"['0679731938', '0394524942', '0385116594']","4,047,732 in Books (","['0525557342', '067976500X', '0894809261', '06...",$5.99,0684815109,100


In [50]:
a = data_kaggle.reset_index()
b = data_amazon_sel.reset_index(drop=True).reset_index()
a.head()

,index,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [51]:
b.head()

,index,category,description,title,also_buy,rank,also_view,price,asin,match_value
0,0,"['Books', 'New, Used &amp; Rental Textbooks', ...","['""A fine introduction to the subject, clear, ...",Classical Mythology,"['019953831X', '0199536325', '0199537968', '05...","5,219,563 in Books (","['019999739X', '0195308050', '0195397703', '00...",NaN,0195143388,100
1,1,"['Books', ""Children's Books"", 'Animals']","['Pamela Duncan Edwards and Henry Cole, the cr...",Clara Caterpillar,[],"4,370,670 in Books (","['1782358854', '0064435024', '0064435164', '00...",$14.89,0060289953,76
2,2,"['Books', 'Literature &amp; Fiction', 'Literary']",['Darlene Biese Schultz teaches fiction writin...,Decision or Destiny,[],"10,562,636 in Books (",[],$12.99,1463527101,72
3,3,"['Books', 'Health, Fitness &amp; Dieting']",[],Flu: The Story of the Great Influenza Pandmic ...,"['0143036491', 'B005APQFD6', '1594489254', '15...","10,931,958 in Books (","['0143036491', '1541736125', '0195307143', '05...",$3.80,0333751051,99
4,4,"['Books', 'Arts & Photography', 'Collections, ...","['This compelling book provides a historical, ...",Mummies of the World,[],"3,982,059 in Books (",[],$29.99,3791350307,76


In [54]:
df_merged = pd.merge(a, b, on='index', how='inner')
df_merged.head()

,index,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,category,description,title,also_buy,rank,also_view,price,asin,match_value
0,0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,"['Books', 'New, Used &amp; Rental Textbooks', ...","['""A fine introduction to the subject, clear, ...",Classical Mythology,"['019953831X', '0199536325', '0199537968', '05...","5,219,563 in Books (","['019999739X', '0195308050', '0195397703', '00...",NaN,0195143388,100
1,1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"['Books', ""Children's Books"", 'Animals']","['Pamela Duncan Edwards and Henry Cole, the cr...",Clara Caterpillar,[],"4,370,670 in Books (","['1782358854', '0064435024', '0064435164', '00...",$14.89,0060289953,76
2,2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,"['Books', 'Literature &amp; Fiction', 'Literary']",['Darlene Biese Schultz teaches fiction writin...,Decision or Destiny,[],"10,562,636 in Books (",[],$12.99,1463527101,72
3,3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,"['Books', 'Health, Fitness &amp; Dieting']",[],Flu: The Story of the Great Influenza Pandmic ...,"['0143036491', 'B005APQFD6', '1594489254', '15...","10,931,958 in Books (","['0143036491', '1541736125', '0195307143', '05...",$3.80,0333751051,99
4,4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,"['Books', 'Arts & Photography', 'Collections, ...","['This compelling book provides a historical, ...",Mummies of the World,[],"3,982,059 in Books (",[],$29.99,3791350307,76


In [64]:
df_merged.iloc[:,[1,2,3,4,5]+list(np.arange(9,df_merged.shape[1], 1))]

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,category,description,title,also_buy,rank,also_view,price,asin,match_value
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,"['Books', 'New, Used &amp; Rental Textbooks', ...","['""A fine introduction to the subject, clear, ...",Classical Mythology,"['019953831X', '0199536325', '0199537968', '05...","5,219,563 in Books (","['019999739X', '0195308050', '0195397703', '00...",NaN,0195143388,100
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,"['Books', ""Children's Books"", 'Animals']","['Pamela Duncan Edwards and Henry Cole, the cr...",Clara Caterpillar,[],"4,370,670 in Books (","['1782358854', '0064435024', '0064435164', '00...",$14.89,0060289953,76
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,"['Books', 'Literature &amp; Fiction', 'Literary']",['Darlene Biese Schultz teaches fiction writin...,Decision or Destiny,[],"10,562,636 in Books (",[],$12.99,1463527101,72
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,"['Books', 'Health, Fitness &amp; Dieting']",[],Flu: The Story of the Great Influenza Pandmic ...,"['0143036491', 'B005APQFD6', '1594489254', '15...","10,931,958 in Books (","['0143036491', '1541736125', '0195307143', '05...",$3.80,0333751051,99
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,"['Books', 'Arts & Photography', 'Collections, ...","['This compelling book provides a historical, ...",Mummies of the World,[],"3,982,059 in Books (",[],$29.99,3791350307,76
5,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,"['Books', 'Crafts, Hobbies & Home', 'Antiques ...",[],Kitchens and Kitchenware,[],"7,078,424 in Books (",[],$6.59,0870694251,70
6,0425176428,What If?: The World's Foremost Military Histor...,Robert Cowley,2000,Berkley Publishing Group,"['Books', 'History', 'Historical Study &amp; E...",['Slight shelf wear. Pages are clean. Binding ...,What If? : Military Historians Imagine What Mi...,"['042518613X', '0425198189', '1616085460', '03...","5,592,796 in Books (","['042518613X', '0399152385', '0425198189', '16...",$2.40,0330487248,84
7,0671870432,PLEADING GUILTY,Scott Turow,1993,Audioworks,"['Books', 'Mystery, Thriller &amp; Suspense', ...","['RUTH RENDELL has won numerous awards, includ...","GUILTY THING SURPRISED, A","['0345513584', '0345498453', '0375704892', '00...","2,025,533 in Books (","['0345342534', '0345345304', '0375704892', '03...",$17.78,0099534843,67
8,0679425608,Under the Black Flag: The Romance and the Real...,David Cordingly,1996,Random House,"['Books', 'Engineering &amp; Transportation', ...","[""David Cordingly is the world's foremost expe...",Under the Black Flag: The Romance and the Real...,"['015603462X', '1479322571', '0312335806', '07...","2,552,673 in Books (","['015603462X', '1631492101', '1479322571', '19...",$10.55,0156005492,92
9,074322678X,Where You'll Find Me: And Other Stories,Ann Beattie,2002,Scribner,"['Books', 'Literature &amp; Fiction', 'Short S...",['Ann Beattie is a writer of enormous compassi...,Where You'Ll Find Me: And Other Stories,"['0679731938', '0394524942', '0385116594']","4,047,732 in Books (","['0525557342', '067976500X', '0894809261', '06...",$5.99,0684815109,100


In [65]:
df_merged.to_csv('merged_dataset.csv')